In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Bidirectional
from attention import AttentionDecoder
# returns train, inference_encoder and inference_decoder models
def define_updated(n_input, n_output, n_units):
    # define training encoder
    encoder_inputs = Input(shape=(None, n_input))
    encoder = Bidirectional(LSTM(n_units,return_sequences=True, return_state=True))
    encoder1 = Bidirectional(LSTM(n_units, return_state=True))
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_outputs, state_h, state_c = encoder1(encoder_outputs, state_h, state_c)
    encoder_states = [state_h, state_c]
    
    # define training decoder
    decoder_inputs = Input(shape=(None, n_output))
    embedding = Embedding(10000, 64)
    decoder_lstm1 = Bidirectional(LSTM(n_units, return_sequences=True, return_state=True))
    decoder_lstm2 = Bidirectional(LSTM(n_units, return_sequences=True, return_state=True))
    attention_decoder = AttentionDecoder(n_units) # !!!!!!!!!  n_features required here. !!!!!!!!!!!!!!
    
    temp = embedding(decoder_inputs)
    temp, _, _ = decoder_lstm1(temp, initial_state=encoder_states)
    decoder_outputs, _, _ = decoder_lstm2(temp, initial_state=encoder_states)
    decoder_outputs = attention_decoder(decoder_outputs)
    decoder_dense = Dense(n_output, activation='softmax')
    
    decoder_outputs = decoder_dense(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    
    # define inference encoder
    encoder_model = Model(encoder_inputs, encoder_states)
    
    # define inference decoder
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    temp = embedding(decoder_inputs)
    temp, _, _ = decoder_lstm1(temp, initial_state=decoder_states_inputs)
    decoder_outputs, state_h, state_c = decoder_lstm2(temp, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
    
    # return all models
    return model, encoder_model, decoder_model